In [1]:
import requests
import time as t
from mpl_toolkits.basemap import Basemap
from shapely.geometry import LineString
from matplotlib import pyplot as plt
import numpy as np
import os
import urllib
import json
%matplotlib inline

# Env vars and functions

In [2]:
def decode(encoded):
    inv = 1.0 / 1e6
    decoded = []
    previous = [0,0]
    i = 0
    while i < len(encoded):
        ll = [0,0]
        for j in [0, 1]:
            shift = 0
            byte = 0x20
            while byte >= 0x20:
                byte = ord(encoded[i]) - 63
                i += 1
                ll[j] |= (byte & 0x1f) << shift
                shift += 5
            ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
            previous[j] = ll[j]
        decoded.append([float('%.6f' % (ll[1] * inv)), float('%.6f' % (ll[0] * inv))])
    return decoded


def synthesize_gps(edges, shape, distribution="normal", stddev=None,uuid='999999'):
    
    jsonDict = {"uuid":"100343", "trace":[]}
    coords = decode(shape)
    sttm = t.time() - 100000
    
    for i, edge in enumerate(edges):
        dist = edge['length']
        speed = edge['speed']
        beginShapeIndex = edge['begin_shape_index']
        endShapeIndex = edge['end_shape_index']
        lon, lat = coords[endShapeIndex]
        if stddev is not None:
            avgLat = np.mean(np.array(coords)[:,1])
            stddevLon = stddev / 111.111     # approx. 111.111 km per deg lon unless very close to the poles
            stddevLat = stddev / (111.111 * np.cos(avgLat))     # approx 111.111 km * cos(lat) per deg lat
            lon += np.random.normal(scale=stddevLon)
            lat += np.random.normal(scale=stddevLat)
        dur = dist / speed * 3600.0
        time = sttm + dur
        time = int(round(time))
        if i == 0:
            st_lon, st_lat = coords[beginShapeIndex]
            jsonDict["trace"].append({"lat":st_lat,"lon":st_lon,"time":time})
        jsonDict["trace"].append({"lat": lat,"lon":lon,"time":time})
        sttm = time

    return jsonDict

# Open Traffic Reporter Validation Service 

### 1. Get New Route

In [29]:
# # 349 38th
# stLat = 37.8261
# stLon = -122.25872

# 2527 24th 
stLat = 37.75286
stLon = -122.40486

endLat = 37.790168
endLon = -122.402264

jsonDict = {"locations":[
        {"lat":stLat,"lon":stLon},
        {"lat":endLat,"lon":endLon}],
            "costing":"auto","costing_options":{"auto":{"country_crossing_penalty":2000.0}},
            "directions_options":{"units":"kilometers"},"id":"my_work_route"}

#### from mapzen.com 

In [31]:
baseUrl = 'https://valhalla.mapzen.com/route'
apiKey = os.environ.get('MAPZEN_API')
payload = {"json": json.dumps(jsonDict), "apiKey": apiKey}
route = requests.get(baseUrl, params=payload)
shape = route.json()['trip']['legs'][0]['shape']
inv = 1.0 / 1e6

In [33]:
route.url

u'https://valhalla.mapzen.com/route?json=%7B%22id%22%3A+%22my_work_route%22%2C+%22costing_options%22%3A+%7B%22auto%22%3A+%7B%22country_crossing_penalty%22%3A+2000.0%7D%7D%2C+%22locations%22%3A+%5B%7B%22lat%22%3A+37.75286%2C+%22lon%22%3A+-122.40486%7D%2C+%7B%22lat%22%3A+37.790168%2C+%22lon%22%3A+-122.402264%7D%5D%2C+%22costing%22%3A+%22auto%22%2C+%22directions_options%22%3A+%7B%22units%22%3A+%22kilometers%22%7D%7D&apiKey=mapzen-fxf7XA5'

In [32]:
route.status_code

200

#### from local valhalla server (via docker)

In [30]:
baseUrl = 'http://localhost:8003/route'
payload = {"json": json.dumps(jsonDict)}
route = requests.get(baseUrl, params=payload)

ConnectionError: ('Connection aborted.', BadStatusLine("''",))

In [24]:
route.url

u'https://valhalla.mapzen.com/route?json=%7B%22id%22%3A+%22my_work_route%22%2C+%22costing_options%22%3A+%7B%22auto%22%3A+%7B%22country_crossing_penalty%22%3A+2000.0%7D%7D%2C+%22locations%22%3A+%5B%7B%22lat%22%3A+37.8261%2C+%22street%22%3A+%2238th%22%2C+%22lon%22%3A+-122.25872%7D%2C+%7B%22lat%22%3A+37.790168%2C+%22street%22%3A+%22New+Montgomery%22%2C+%22lon%22%3A+-122.402264%7D%5D%2C+%22costing%22%3A+%22auto%22%2C+%22directions_options%22%3A+%7B%22units%22%3A+%22kilometers%22%7D%7D&apiKey=mapzen-fxf7XA5'

### 2. Pass Route Shape to Map Matching API 

In [59]:
baseUrl = 'https://valhalla.mapzen.com/trace_attributes?'
apiKey = os.environ.get('MAPZEN_API')
url = baseUrl + 'api_key=' + apiKey
payload = {
    "encoded_polyline": shape,
    "costing": "pedestrian",
    "directions_options": {
        "units": "kilometers"
    },
    "shape_match": "map_snap",
    "trace_options": {
        "turn_penalty_factor": 500
    }
}
matched = requests.post(url, json=payload)

In [50]:
print 'Matched {0} points. Showing the first below: \n{1}'.format(
    len(matched.json()['matched_points']), matched.json()['matched_points'][0])

Matched 313 points. Showing the first below: 
{u'lon': -122.258926, u'distance_from_trace_point': 0.0, u'distance_along_edge': 0.119, u'type': u'matched', u'lat': 37.82618, u'edge_index': 0}


In [9]:
edges = matched.json()['edges']
print 'Got {0} edges. Showing the first below: \n {1}'.format(len(edges), edges[0])

Got 153 edges. Showing the first below: 
 {u'drive_on_right': True, u'max_downward_grade': -2, u'end_heading': 206, u'surface': u'paved_smooth', u'names': [u'Manila Avenue'], u'end_shape_index': 1, u'way_id': 307501648, u'speed': 22, u'id': 325550987826, u'pedestrian_type': u'foot', u'use': u'road', u'density': 14, u'begin_shape_index': 0, u'max_upward_grade': 0, u'traversability': u'both', u'lane_count': 0, u'travel_mode': u'pedestrian', u'mean_elevation': 72, u'weighted_grade': -1.667, u'end_node': {u'elapsed_time': 116, u'type': u'street_intersection', u'time_zone': u'PST-08PDT+01,M3.2.0/02:00,M11.1.0/02:00', u'admin_index': 0}, u'road_class': u'residential', u'begin_heading': 206, u'bicycle_network': 0, u'length': 0.102}


### 3. Create Fake GPS traces

#### Extract coords from routes.shape, perturb, and save to map-matched json

In [104]:
jsonDict = synthesize_gps(edges, shape, stddev=0.05)   # stddev in km

In [105]:
print 'Sending {0} fake GPS measurements to the reporter'.format(len(jsonDict['trace']))

Sending 154 fake GPS measurements to the reporter


### 4. Send Fake GPS to Reporter

In [106]:
baseUrl = 'http://localhost:8002/report'
payload = {"json": json.dumps(jsonDict)} 
report = requests.get(baseUrl, params=payload)

In [107]:
segments = report.json()['segments']
print 'Reporter matched {0} GPS measurements to {1} OSMLR segments. Showing the first below: \n{2}'.format(
    len(jsonDict['trace']),len(segments), report.json()['segments'][13])

Reporter matched 154 GPS measurements to 14 OSMLR segments. Showing the first below: 
{u'segment_id': 78383175584, u'begin_shape_index': 144, u'start_time': 1495397954.0, u'length': 146, u'end_time': 1495397973.0, u'end_shape_index': 149}
